In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD, Nadam, RMSprop
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.regularizers import l1, l2

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from models.hdnn import hdnn

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmp9x13bn/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmp9x13bn/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)
C:\Users\Tanuj\Anaconda2\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
lr=0.005
l1 = 0.00001
l2 = 0.00001
dropout = 0.5
n_epochs=100
batch_size=32
input_shape=(140, 140, 3)

name = 'hdnn_relu_140_rgb_corrected_lr_%f_sgd_he_normal__l1_%f_l2_%f_dropout_%f_r' % (lr, l1, l2, dropout)

In [3]:
print('loading model...')
# model = cnn(input_shape=input_shape, init='he_normal')
model = hdnn(input_shape=input_shape, activation_fn='relu', init='he_normal', l1=l1, l2=l2)
model.summary()

optimizer = SGD(lr=lr, clipnorm=4., nesterov=True)
# optimizer = Nadam(lr=lr)
# optimizer = RMSprop(lr=lr)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

csv_logger = CSVLogger('%s_training.log' % name, append=True)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_current.hdf5" % name), verbose=0)

loading model...
Compiling conv2d layer: stage 1 block 1...
Compiling conv2d layer: stage 2 block 1...
Compiling conv2d layer: stage 3 block 1...
Creating hybrid block with 3 conv2d blocks: stage 4
Compiling conv2d layer: stage 4 block 1...
Compiling conv2d layer: stage 4 block 2...
Compiling conv2d layer: stage 4 block 3...
Merging hybrid blocks : stage 4
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 140, 140, 3)   0                                            
____________________________________________________________________________________________________
C1_1 (Convolution2D)             (None, 134, 134, 84)  12432       input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (N

In [4]:
print('Initializing data generators...')
train_set_file = config.corrected_train_data_file
validation_set_file = config.corrected_validation_data_file
test_set_file = config.corrected_test_data_file

train_data_gen = DataGenerator(dataset_file=train_set_file, batch_size=batch_size)
validation_data_gen = DataGenerator(dataset_file=validation_set_file, batch_size=batch_size)
test_data_gen = DataGenerator(dataset_file=test_set_file, batch_size=batch_size)
print('done.')

Initializing data generators...
done.


In [ ]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, best_model_checkpointer, current_model_checkpointer],
                              nb_worker=2)
print('done.')

Fitting model...


DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpsik4vu/5d8a7bab200dc8090d4573cde3b5cd46.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpsik4vu/5d8a7bab200dc8090d4573cde3b5cd46.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpykv5_x/231730f5ff4cdd4403902aa79e1ec083.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpykv5_x/231730f5ff4cdd4403902aa79e1ec083.exp



Epoch 1/100
 5856/10496 [===============>..............] - ETA: 49s - loss: 0.7030 - acc: 0.6557

In [ ]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])